In [2]:
import re
import pandas as pd

# # Opcional: para remover stopwords
# import nltk
# from nltk.corpus import stopwords
# nltk.download('stopwords') # Execute se necessário
# stop_words = set(stopwords.words('english')) # Defina a lista de stopwords (ajuste o idioma se necessário)

# Função para limpar o texto do comentário
def clean_text(text):
    if isinstance(text, str):
        # Converter para minúsculas
        text = text.lower()
        # Remover pontuação, números e caracteres especiais (mantém apenas letras e espaços)
        #text = re.sub(r'[^a-z\s]', '', text)
        # Opcional: Remover stopwords
        #text = ' '.join(word for word in text.split() if word not in stop_words)
        # Substituir múltiplos espaços por um único espaço e remover espaços no início/fim
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return "" # Retorna string vazia ou algum marcador para entradas não-string/NaN

# --- Processar dataset_train.csv ---
train_file_path = 'dataset_train.csv'

try:
    df_train = pd.read_csv(train_file_path, sep='|')

    # Verificar o nome da coluna de comentários. Baseado nos passos anteriores, assumimos que seja 'comment'.
    # Se for outro nome, ajuste aqui:
    comment_column_name = 'comment' # <--- Verifique o nome real da coluna no seu CSV

    # Verificar se a coluna existe
    if comment_column_name not in df_train.columns:
        raise ValueError(f"Coluna '{comment_column_name}' não encontrada em {train_file_path}")

    # Tratar possíveis valores nulos na coluna de comentários ANTES de limpar
    # É crucial que a limpeza receba strings. Remover ou preencher NaNs é importante.
    print(f"Valores nulos na coluna '{comment_column_name}' antes da limpeza no treino: {df_train[comment_column_name].isnull().sum()}")
    df_train.dropna(subset=[comment_column_name], inplace=True) # Remover linhas com comentários nulos
    print(f"Shape do dataframe de treino após remover nulos: {df_train.shape}")


    # Aplicar a função de limpeza à coluna de comentários
    # Vamos criar uma nova coluna para manter o texto original, se desejar
    df_train['comment_cleaned'] = df_train[comment_column_name].apply(clean_text)

    print("\nDataFrame de Treino com Comentários Limpos (Primeiras Linhas):")
    print(df_train.head())


except FileNotFoundError:
    print(f"Erro: Arquivo '{train_file_path}' não encontrado.")
except ValueError as ve:
    print(f"Erro de validação: {ve}")
except Exception as e:
    print(f"Ocorreu um erro ao processar {train_file_path}: {e}")


# --- Processar dataset_valid.csv ---
valid_file_path = 'dataset_valid.csv'

try:
    df_valid = pd.read_csv(valid_file_path, sep='|')

    # Verificar o nome da coluna de comentários no arquivo de validação
    # (Assumimos que é o mesmo nome da coluna de treino)
    comment_column_name_valid = 'comment' # <--- Verifique se é o mesmo nome no seu CSV de validação

    # Verificar se a coluna existe
    if comment_column_name_valid not in df_valid.columns:
         raise ValueError(f"Coluna '{comment_column_name_valid}' não encontrada em {valid_file_path}")

    # Tratar possíveis valores nulos na coluna de comentários ANTES de limpar
    print(f"\nValores nulos na coluna '{comment_column_name_valid}' antes da limpeza na validação: {df_valid[comment_column_name_valid].isnull().sum()}")
    df_valid.dropna(subset=[comment_column_name_valid], inplace=True) # Remover linhas com comentários nulos
    print(f"Shape do dataframe de validação após remover nulos: {df_valid.shape}")

    # Aplicar a função de limpeza à coluna de comentários
    df_valid['comment_cleaned'] = df_valid[comment_column_name_valid].apply(clean_text)

    print("\nDataFrame de Validação com Comentários Limpos (Primeiras Linhas):")
    print(df_valid.head())

    

except FileNotFoundError:
    print(f"Erro: Arquivo '{valid_file_path}' não encontrado.")
except ValueError as ve:
    print(f"Erro de validação: {ve}")
except Exception as e:
    print(f"Ocorreu um erro ao processar {valid_file_path}: {e}")

Valores nulos na coluna 'comment' antes da limpeza no treino: 0
Shape do dataframe de treino após remover nulos: (651, 1)

DataFrame de Treino com Comentários Limpos (Primeiras Linhas):
                                             comment  \
0  judging from previous posts this used to be a ...   
1  we , there were four of us , arrived at noon -...   
2  they never brought us complimentary noodles , ...   
3  the food was lousy - too sweet or too salty an...   
4  after all that , they complained to me about t...   

                                     comment_cleaned  
0  judging from previous posts this used to be a ...  
1  we , there were four of us , arrived at noon -...  
2  they never brought us complimentary noodles , ...  
3  the food was lousy - too sweet or too salty an...  
4  after all that , they complained to me about t...  

Valores nulos na coluna 'comment' antes da limpeza na validação: 0
Shape do dataframe de validação após remover nulos: (199, 1)

DataFrame de Vali

In [3]:
# Agora o df_train tem uma coluna 'comment_cleaned' pronta para vetorização/análise
# Você pode salvar este DataFrame limpo, talvez em Parquet
df_train.to_parquet('dataset_train_cleaned.parquet', index=False)
print("\nDataFrame de treino limpo salvo em 'dataset_train_cleaned.parquet'")


DataFrame de treino limpo salvo em 'dataset_train_cleaned.parquet'


In [4]:
s = 'the pizza was delivered cold and the cheese was n t even fully melted !'

In [5]:
import re
print(re.sub(r'\s+', ' ', s).strip())

the pizza was delivered cold and the cheese was n t even fully melted !


In [6]:
re.sub(r'[^a-z\s\']', '', s) # Mantém letras, espaços e apóstrofos
        

'the pizza was delivered cold and the cheese was n t even fully melted '

In [7]:
re.sub(r'\s+n\s+t\s+', " n't ", s)
        

"the pizza was delivered cold and the cheese was n't even fully melted !"

In [8]:
# Agora o df_valid tem uma coluna 'comment_cleaned' pronta para predição
# Você pode salvar este DataFrame limpo, talvez em Parquet
df_valid.to_parquet('dataset_valid_cleaned.parquet', index=False)
print("\nDataFrame de validação limpo salvo em 'dataset_valid_cleaned.parquet'")



DataFrame de validação limpo salvo em 'dataset_valid_cleaned.parquet'


In [9]:
df_train.to_csv('dataset_train_inputs.csv')
df_valid.to_csv('dataset_valid_inputs.csv')